In [1]:
import pydap.client
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
End_Year = 2008
Start_Year = 2000

In [2]:
url_dict = {}
url_dict['Precip']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_'
url_dict['Rhumiditymax']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_'
url_dict['Rhumiditymin']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmin/rmin_'
url_dict['Shumidit']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/sph/sph_'
url_dict['Sradiation']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_'
url_dict['WindFromDir']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/th/th_'
url_dict['WindVelocity']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_'
url_dict['Tempmax']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmx/tmmx_'
url_dict['Tempmin']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmn/tmmn_'
url_dict['BurningIndex']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/bi/bi_'
url_dict['FuelMoist100']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/fm100/fm100_'
url_dict['FuelMoist1000']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/fm1000/fm1000_'
url_dict['EnergyRelease']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/erc/erc_'
# url_dict['DroughtSeverity']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pdsi/pdsi_'
url_dict['EvapoTranspiration']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/etr/etr_'
url_dict['EvapoTranspir_G']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pet/pet_'
url_dict['VaporPressDeficit']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vpd/vpd_'

In [3]:
base_url_part = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_'
url = base_url_part + str(End_Year) + '.nc'
dataset = pydap.client.open_url(url)

In [4]:
#Slice the Bounding Box we wanted here.
lon_ = dataset['lon']
lon = lon_[:].data
lat_ = dataset['lat']
lat = lat_[:].data

import numpy as np
SD_county_lats = np.where((lat<33.6) & (lat>32.4))[0]
SD_county_lons = np.where((lon<-116.0) & (lon>-118))[0]

lat.shape, lon.shape

((585,), (1386,))

In [5]:
lat_begin, lat_end = SD_county_lats[0], SD_county_lats[-1]
lon_begin, lon_end = SD_county_lons[0], SD_county_lons[-1]
lat_begin, lat_end, lon_begin, lon_end 

(380, 408, 163, 210)

In [6]:
flag=0
final_df = pd.DataFrame()
for Year in tqdm(range(End_Year,Start_Year-1,-1)):
    print("Downloading data for year {}".format(Year))
    url = base_url_part + str(Year) + '.nc'
    dataset = pydap.client.open_url(url)
    #Create the Base DataFrame with Days and Co-ordinates
    days = pd.to_timedelta(dataset['day'].data[:], unit='days')
    days = days + pd.to_datetime("19000101", format="%Y%m%d")
    index = pd.MultiIndex.from_product([days.values,
                                        lat[SD_county_lats].astype('f8'),
                                        lon[SD_county_lons].astype('f8')],
                                       names=['date', 'latitude', 'longitude'])
    base_df = pd.DataFrame(index=index) 
    ##Load each Attriubute below
    for key, item in tqdm(url_dict.items()):
        print ("Processing for", item)
        url = item + str(Year) + '.nc'
#         print (url)
        dataset = pydap.client.open_url(url)
        for column in dataset.keys():
            if column in ('lat', 'lon', 'crs', 'day'):
                continue
            #Here is where the Data Download happens
            attrs = dataset[column].attributes
            missing_value, scale, bias = attrs['missing_value'], attrs['scale_factor'], attrs['add_offset']
            print("'{}': missing_values: {}, scale: {}, bias: {}".format(column, missing_value, scale, bias))
            raw_data = dataset[column][:, lat_begin:lat_end, lon_begin:lon_end]
            base_date = pd.to_datetime("19000101", format="%Y%m%d")
            values, days, lats, lons = raw_data.data
            print("Data downloaded")
            days = base_date + pd.to_timedelta(days, unit="days")
            col_name = column + "_" + dataset[column].units
            if key == 'Tempmax':
                col_name = "max_" + col_name
            elif key == 'Tempmin':
                col_name = "min_" + col_name
            values = values.astype('d')
            values[values == missing_value] = np.nan
            values = values * scale + bias
            base_df.loc[(days, lats, lons), col_name] = values.ravel()
    display(base_df.head(3))
    base_df.to_parquet("gridMet_{}.parquet.gz".format(Year), compression='gzip')
    final_df = pd.concat([final_df, base_df])

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_
'precipitation_amount': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmin/rmin_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/sph/sph_
'specific_humidity': missing_values: 32767, scale: 1e-05, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_
'surface_downwelling_shortwave_flux_in_air': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/th/th_
'wind_from_direction': missing_values: 32767, sca

precipitation_amount_mm  \
date       latitude  longitude                              
2008-01-01 33.566667 -117.975000                      0.0   
                     -117.933333                      0.0   
                     -117.891667                      0.0   

                                  relative_humidity_%  \
date       latitude  longitude                          
2008-01-01 33.566667 -117.975000                 30.3   
                     -117.933333                 30.0   
                     -117.891667                 28.9   

                                  specific_humidity_kg/kg  \
date       latitude  longitude                              
2008-01-01 33.566667 -117.975000                  0.00434   
                     -117.933333                  0.00434   
                     -117.891667                  0.00429   

                                  surface_downwelling_shortwave_flux_in_air_W m-2  \
date       latitude  longitude                                                      
2008-01-01 33.566667 -117.975000                                            127.3   
                     -117.933333                                            126.8   
                     -117.891667                                            127.4   

                                  wind_from_direction_Degrees Clockwise from north  \
date       latitude  longitude                                                       
2008-01-01 33.566667 -117.975000                                              61.0   
                     -117.933333                                              61.0   
                     -117.891667                                              61.0   

                                  wind_speed_m/s  max_air_temperature_K  \
date       latitude  longitude                                            
2008-01-01 33.566667 -117.975000             4.3                  294.0   
                     -117.933333             4.4                  293.7   
                     -117.891667             4.5                  293.6   

                                  min_air_temperature_K  \
date       latitude  longitude                            
2008-01-01 33.566667 -117.975000                  284.7   
                     -117.933333                  284.7   
                     -117.891667                  284.8   

                                  burning_index_g_Unitless  \
date       latitude  longitude                               
2008-01-01 33.566667 -117.975000                      42.0   
                     -117.933333                      42.0   
                     -117.891667                      43.0   

                                  dead_fuel_moisture_100hr_Percent  \
date       latitude  longitude                                       
2008-01-01 33.566667 -117.975000                              12.6   
                     -117.933333                              12.6   
                     -117.891667                              12.5   

                                  dead_fuel_moisture_1000hr_Percent  \
date       latitude  longitude                                        
2008-01-01 33.566667 -117.975000                               13.6   
                     -117.933333                               13.9   
                     -117.891667                               13.9   

                                  energy_release_component-g_Unitless  \
date       latitude  longitude                                          
2008-01-01 33.566667 -117.975000                                 45.0   
                     -117.933333                                 44.0   
                     -117.891667                                 44.0   

                                  potential_evapotranspiration_mm  \
date       latitude  longitude                                      
2008-01-01 33.566667 -117.975000                              3.6   
                

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_
'precipitation_amount': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmin/rmin_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/sph/sph_
'specific_humidity': missing_values: 32767, scale: 1e-05, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_
'surface_downwelling_shortwave_flux_in_air': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/th/th_
'wind_from_direction': missing_values: 32767, sca

precipitation_amount_mm  \
date       latitude  longitude                              
2007-01-01 33.566667 -117.975000                      0.0   
                     -117.933333                      0.0   
                     -117.891667                      0.0   

                                  relative_humidity_%  \
date       latitude  longitude                          
2007-01-01 33.566667 -117.975000                 34.6   
                     -117.933333                 35.4   
                     -117.891667                 34.6   

                                  specific_humidity_kg/kg  \
date       latitude  longitude                              
2007-01-01 33.566667 -117.975000                  0.00424   
                     -117.933333                  0.00425   
                     -117.891667                  0.00419   

                                  surface_downwelling_shortwave_flux_in_air_W m-2  \
date       latitude  longitude                                                      
2007-01-01 33.566667 -117.975000                                            140.8   
                     -117.933333                                            139.8   
                     -117.891667                                            138.2   

                                  wind_from_direction_Degrees Clockwise from north  \
date       latitude  longitude                                                       
2007-01-01 33.566667 -117.975000                                             307.0   
                     -117.933333                                             307.0   
                     -117.891667                                             307.0   

                                  wind_speed_m/s  max_air_temperature_K  \
date       latitude  longitude                                            
2007-01-01 33.566667 -117.975000             3.4                  290.7   
                     -117.933333             3.3                  290.5   
                     -117.891667             3.1                  290.5   

                                  min_air_temperature_K  \
date       latitude  longitude                            
2007-01-01 33.566667 -117.975000                  280.4   
                     -117.933333                  280.4   
                     -117.891667                  280.2   

                                  burning_index_g_Unitless  \
date       latitude  longitude                               
2007-01-01 33.566667 -117.975000                      39.0   
                     -117.933333                      39.0   
                     -117.891667                      39.0   

                                  dead_fuel_moisture_100hr_Percent  \
date       latitude  longitude                                       
2007-01-01 33.566667 -117.975000                              10.9   
                     -117.933333                              10.9   
                     -117.891667                              10.9   

                                  dead_fuel_moisture_1000hr_Percent  \
date       latitude  longitude                                        
2007-01-01 33.566667 -117.975000                               12.4   
                     -117.933333                               12.3   
                     -117.891667                               12.2   

                                  energy_release_component-g_Unitless  \
date       latitude  longitude                                          
2007-01-01 33.566667 -117.975000                                 50.0   
                     -117.933333                                 50.0   
                     -117.891667                                 50.0   

                                  potential_evapotranspiration_mm  \
date       latitude  longitude                                      
2007-01-01 33.566667 -117.975000                              2.5   
                

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_
'precipitation_amount': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmin/rmin_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/sph/sph_
'specific_humidity': missing_values: 32767, scale: 1e-05, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_
'surface_downwelling_shortwave_flux_in_air': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/th/th_
'wind_from_direction': missing_values: 32767, sca

...

precipitation_amount_mm  \
date       latitude  longitude                              
2002-01-01 33.566667 -117.975000                      1.0   
                     -117.933333                      1.1   
                     -117.891667                      0.6   

                                  relative_humidity_%  \
date       latitude  longitude                          
2002-01-01 33.566667 -117.975000                 52.9   
                     -117.933333                 53.3   
                     -117.891667                 52.4   

                                  specific_humidity_kg/kg  \
date       latitude  longitude                              
2002-01-01 33.566667 -117.975000                  0.00696   
                     -117.933333                  0.00692   
                     -117.891667                  0.00680   

                                  surface_downwelling_shortwave_flux_in_air_W m-2  \
date       latitude  longitude                                                      
2002-01-01 33.566667 -117.975000                                            115.9   
                     -117.933333                                            115.0   
                     -117.891667                                            114.3   

                                  wind_from_direction_Degrees Clockwise from north  \
date       latitude  longitude                                                       
2002-01-01 33.566667 -117.975000                                             199.0   
                     -117.933333                                             199.0   
                     -117.891667                                             199.0   

                                  wind_speed_m/s  max_air_temperature_K  \
date       latitude  longitude                                            
2002-01-01 33.566667 -117.975000             0.7                  291.3   
                     -117.933333             0.7                  291.1   
                     -117.891667             0.8                  291.2   

                                  min_air_temperature_K  \
date       latitude  longitude                            
2002-01-01 33.566667 -117.975000                  283.0   
                     -117.933333                  283.3   
                     -117.891667                  283.1   

                                  burning_index_g_Unitless  \
date       latitude  longitude                               
2002-01-01 33.566667 -117.975000                       0.0   
                     -117.933333                       0.0   
                     -117.891667                       0.0   

                                  dead_fuel_moisture_100hr_Percent  \
date       latitude  longitude                                       
2002-01-01 33.566667 -117.975000                              19.7   
                     -117.933333                              19.2   
                     -117.891667                              18.7   

                                  dead_fuel_moisture_1000hr_Percent  \
date       latitude  longitude                                        
2002-01-01 33.566667 -117.975000                               19.8   
                     -117.933333                               19.1   
                     -117.891667                               18.3   

                                  energy_release_component-g_Unitless  \
date       latitude  longitude                                          
2002-01-01 33.566667 -117.975000                                 13.0   
                     -117.933333                                 15.0   
                     -117.891667                                 17.0   

                                  potential_evapotranspiration_mm  \
date       latitude  longitude                                      
2002-01-01 33.566667 -117.975000                              1.1   
                

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_
'precipitation_amount': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmin/rmin_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/sph/sph_
'specific_humidity': missing_values: 32767, scale: 1e-05, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_
'surface_downwelling_shortwave_flux_in_air': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/th/th_
'wind_from_direction': missing_values: 32767, sca

precipitation_amount_mm  \
date       latitude  longitude                              
2001-01-01 33.566667 -117.975000                      0.0   
                     -117.933333                      0.0   
                     -117.891667                      0.0   

                                  relative_humidity_%  \
date       latitude  longitude                          
2001-01-01 33.566667 -117.975000                 51.9   
                     -117.933333                 52.8   
                     -117.891667                 51.0   

                                  specific_humidity_kg/kg  \
date       latitude  longitude                              
2001-01-01 33.566667 -117.975000                  0.00599   
                     -117.933333                  0.00598   
                     -117.891667                  0.00586   

                                  surface_downwelling_shortwave_flux_in_air_W m-2  \
date       latitude  longitude                                                      
2001-01-01 33.566667 -117.975000                                            108.0   
                     -117.933333                                            106.9   
                     -117.891667                                            106.6   

                                  wind_from_direction_Degrees Clockwise from north  \
date       latitude  longitude                                                       
2001-01-01 33.566667 -117.975000                                             131.0   
                     -117.933333                                             131.0   
                     -117.891667                                             131.0   

                                  wind_speed_m/s  max_air_temperature_K  \
date       latitude  longitude                                            
2001-01-01 33.566667 -117.975000             1.7                  289.2   
                     -117.933333             1.7                  289.0   
                     -117.891667             1.7                  289.0   

                                  min_air_temperature_K  \
date       latitude  longitude                            
2001-01-01 33.566667 -117.975000                  280.7   
                     -117.933333                  280.8   
                     -117.891667                  280.8   

                                  burning_index_g_Unitless  \
date       latitude  longitude                               
2001-01-01 33.566667 -117.975000                      26.0   
                     -117.933333                      26.0   
                     -117.891667                      27.0   

                                  dead_fuel_moisture_100hr_Percent  \
date       latitude  longitude                                       
2001-01-01 33.566667 -117.975000                              14.9   
                     -117.933333                              14.7   
                     -117.891667                              14.0   

                                  dead_fuel_moisture_1000hr_Percent  \
date       latitude  longitude                                        
2001-01-01 33.566667 -117.975000                               13.2   
                     -117.933333                               13.2   
                     -117.891667                               12.7   

                                  energy_release_component-g_Unitless  \
date       latitude  longitude                                          
2001-01-01 33.566667 -117.975000                                 39.0   
                     -117.933333                                 39.0   
                     -117.891667                                 41.0   

                                  potential_evapotranspiration_mm  \
date       latitude  longitude                                      
2001-01-01 33.566667 -117.975000                              1.3   
                

Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_
'precipitation_amount': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmin/rmin_
'relative_humidity': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/sph/sph_
'specific_humidity': missing_values: 32767, scale: 1e-05, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_
'surface_downwelling_shortwave_flux_in_air': missing_values: 32767, scale: 0.1, bias: 0.0
Data downloaded
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/th/th_
'wind_from_direction': missing_values: 32767, sca

precipitation_amount_mm  \
date       latitude  longitude                              
2000-01-01 33.566667 -117.975000                      0.0   
                     -117.933333                      0.0   
                     -117.891667                      0.0   

                                  relative_humidity_%  \
date       latitude  longitude                          
2000-01-01 33.566667 -117.975000                 40.5   
                     -117.933333                 40.9   
                     -117.891667                 41.2   

                                  specific_humidity_kg/kg  \
date       latitude  longitude                              
2000-01-01 33.566667 -117.975000                  0.00600   
                     -117.933333                  0.00593   
                     -117.891667                  0.00580   

                                  surface_downwelling_shortwave_flux_in_air_W m-2  \
date       latitude  longitude                                                      
2000-01-01 33.566667 -117.975000                                            139.7   
                     -117.933333                                            136.6   
                     -117.891667                                            133.2   

                                  wind_from_direction_Degrees Clockwise from north  \
date       latitude  longitude                                                       
2000-01-01 33.566667 -117.975000                                             222.0   
                     -117.933333                                             222.0   
                     -117.891667                                             222.0   

                                  wind_speed_m/s  max_air_temperature_K  \
date       latitude  longitude                                            
2000-01-01 33.566667 -117.975000             2.1                  292.0   
                     -117.933333             2.1                  291.9   
                     -117.891667             2.3                  292.1   

                                  min_air_temperature_K  \
date       latitude  longitude                            
2000-01-01 33.566667 -117.975000                  282.2   
                     -117.933333                  282.2   
                     -117.891667                  282.1   

                                  burning_index_g_Unitless  \
date       latitude  longitude                               
2000-01-01 33.566667 -117.975000                      31.0   
                     -117.933333                      32.0   
                     -117.891667                      33.0   

                                  dead_fuel_moisture_100hr_Percent  \
date       latitude  longitude                                       
2000-01-01 33.566667 -117.975000                              12.3   
                     -117.933333                              12.2   
                     -117.891667                              11.8   

                                  dead_fuel_moisture_1000hr_Percent  \
date       latitude  longitude                                        
2000-01-01 33.566667 -117.975000                               12.1   
                     -117.933333                               12.0   
                     -117.891667                               11.5   

                                  energy_release_component-g_Unitless  \
date       latitude  longitude                                          
2000-01-01 33.566667 -117.975000                                 48.0   
                     -117.933333                                 48.0   
                     -117.891667                                 50.0   

                                  potential_evapotranspiration_mm  \
date       latitude  longitude                                      
2000-01-01 33.566667 -117.975000                              1.8   
                

In [7]:
final_df.describe()

,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa
count,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06,2.926320e+06
mean,8.166745e-01,2.960338e+01,6.410530e-03,2.386295e+02,2.270238e+02,3.006786e+00,2.975043e+02,2.830262e+02,3.659815e+01,1.221477e+01,1.292307e+01,4.949300e+01,4.337437e+00,1.344393e+00
std,4.105681e+00,1.649074e+01,2.844963e-03,8.090488e+01,8.166953e+01,1.253823e+00,7.808502e+00,6.125961e+00,1.933930e+01,4.559093e+00,4.145687e+00,2.103502e+01,1.959895e+00,9.760103e-01
min,0.000000e+00,1.000000e-01,3.500000e-04,3.090000e+01,0.000000e+00,4.000000e-01,2.696000e+02,2.601000e+02,0.000000e+00,2.100000e+00,3.500000e+00,0.000000e+00,3.000000e-01,0.000000e+00
25%,0.000000e+00,1.590000e+01,4.300000e-03,1.627000e+02,2.020000e+02,2.200000e+00,2.919000e+02,2.786000e+02,2.600000e+01,8.400000e+00,9.700000e+00,3.300000e+01,2.700000e+00,6.300000e-01
50%,0.000000e+00,2.630000e+01,6.110000e-03,2.474000e+02,2.420000e+02,2.800000e+00,2.967000e+02,2.829000e+02,3.600000e+01,1.230000e+01,1.300000e+01,4.700000e+01,4.200000e+00,1.060000e+00
75%,0.000000e+00,4.150000e+01,8.280000e-03,3.141000e+02,2.780000e+02,3.500000e+00,3.031000e+02,2.876000e+02,5.000000e+01,1.560000e+01,1.580000e+01,6.500000e+01,5.800000e+00,1.810000e+00
max,2.871000e+02,1.000000e+02,2.129000e-02,3.777000e+02,3.590000e+02,1.620000e+01,3.220000e+02,3.072000e+02,1.470000e+02,2.930000e+01,2.820000e+01,1.090000e+02,1.260000e+01,6.660000e+00


In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4576896 entries, (Timestamp('2008-01-01 00:00:00'), 33.56666666666667, -117.97499996666667) to (Timestamp('2000-12-31 00:00:00'), 32.400000000000006, -116.01666663333334)
Data columns (total 14 columns):
 #   Column                                            Dtype  
---  ------                                            -----  
 0   precipitation_amount_mm                           float64
 1   relative_humidity_%                               float64
 2   specific_humidity_kg/kg                           float64
 3   surface_downwelling_shortwave_flux_in_air_W m-2   float64
 4   wind_from_direction_Degrees Clockwise from north  float64
 5   wind_speed_m/s                                    float64
 6   max_air_temperature_K                             float64
 7   min_air_temperature_K                             float64
 8   burning_index_g_Unitless                          float64
 9   dead_fuel_moisture_100hr_Percent                

In [9]:
# %%time
# final_df.to_parquet("gridMet.parquet.gz", compression='gzip')

In [10]:
final_df[final_df.isna()]

precipitation_amount_mm  \
date       latitude  longitude                              
2008-01-01 33.566667 -117.975000                      NaN   
                     -117.933333                      NaN   
                     -117.891667                      NaN   
                     -117.850000                      NaN   
                     -117.808333                      NaN   
...                                                   ...   
2000-12-31 32.400000 -116.183333                      NaN   
                     -116.141667                      NaN   
                     -116.100000                      NaN   
                     -116.058333                      NaN   
                     -116.016667                      NaN   

                                  relative_humidity_%  \
date       latitude  longitude                          
2008-01-01 33.566667 -117.975000                  NaN   
                     -117.933333                  NaN   
                     -117.891667                  NaN   
                     -117.850000                  NaN   
                     -117.808333                  NaN   
...                                               ...   
2000-12-31 32.400000 -116.183333                  NaN   
                     -116.141667                  NaN   
                     -116.100000                  NaN   
                     -116.058333                  NaN   
                     -116.016667                  NaN   

                                  specific_humidity_kg/kg  \
date       latitude  longitude                              
2008-01-01 33.566667 -117.975000                      NaN   
                     -117.933333                      NaN   
                     -117.891667                      NaN   
                     -117.850000                      NaN   
                     -117.808333                      NaN   
...                                                   ...   
2000-12-31 32.400000 -116.183333                      NaN   
                     -116.141667                      NaN   
                     -116.100000                      NaN   
                     -116.058333                      NaN   
                     -116.016667                      NaN   

                                  surface_downwelling_shortwave_flux_in_air_W m-2  \
date       latitude  longitude                                                      
2008-01-01 33.566667 -117.975000                                              NaN   
                     -117.933333                                              NaN   
                     -117.891667                                              NaN   
                     -117.850000                                              NaN   
                     -117.808333                                              NaN   
...                                                                           ...   
2000-12-31 32.400000 -116.183333                                              NaN   
                     -116.141667                                              NaN   
                     -116.100000                                              NaN   
                     -116.058333                                              NaN   
                     -116.016667                                              NaN   

                                  wind_from_direction_Degrees Clockwise from north  \
date       latitude  longitude                                                       
2008-01-01 33.566667 -117.975000                                               NaN   
                     -117.933333                                               NaN   
                     -117.891667                                               NaN   
                     -117.850000                                               NaN   
                     -117.808333                                               NaN